# 单向匹配

## 单向匹配 录入男性评分，女性评分，拒绝列表

In [65]:
import pandas as pd
import numpy as np
import time

# 读取女性评分
with open("Scores_for_Women.txt", "r") as f:
    w_to_m_data = f.readlines()[1:]

# 转化为矩阵 w_to_m_score_matrix[i][j] 为 女性i对男性j的评分
w_to_m_Scores = []    
for i in range(len(w_to_m_data)):
    w_to_m_Scores.append([float(ii) for ii in w_to_m_data[i].strip('\n').split(' ')[:-1]])
    
w_to_m_score_matrix = np.array(w_to_m_Scores)

# 读取女性评分
with open("Scores_for_Men.txt", "r") as f:
    m_to_w_data = f.readlines()[1:]

# 转化为矩阵 m_to_w_score_matrix[i][j] 为 男性i对女性j的评分    
m_to_w_Scores = []    
for i in range(len(m_to_w_data)):
    m_to_w_Scores.append([float(ii) for ii in m_to_w_data[i].strip('\n').split(' ')[:-1]])
    
m_to_w_score_matrix = np.array(m_to_w_Scores)


# 读取拒绝列表
with open("Reject_List.txt", "r") as f:
    reject_data = f.readlines()[1:]

# 转化为列表 
reject_list = []    
for i in range(len(reject_data)):
    reject_list.append([int(ii) for ii in reject_data[i].strip('\n').split(' ')])
    
# print('reject_list', reject_list)

W_dict = {} # 女性可接受词典
M_dict = {} # 男性可接受词典

for i in range(68):
    W_dict[i] = [i for i in range(68)]
    M_dict[i] = [i for i in range(68)]

for j in reject_list: # 按拒绝列表移除不接受的用户
    W_dict[j[0]].remove(j[1])
    M_dict[j[1]].remove(j[0])


Exception ignored in: <function SolverGurobi.__del__ at 0x1183c4dc0>
Traceback (most recent call last):
  File "/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/gurobi.py", line 431, in __del__
    if self._ownsModel:
AttributeError: 'SolverGurobi' object has no attribute '_ownsModel'


FileNotFoundError: [Errno 2] No such file or directory: 'Scores_for_Women.txt'

# One Side Matching

## 在此规则中，以男性匹配为例，同一名女性用户最多可以被推荐X次，一名男性最多只能收获一名推荐名额，目标1.最大化权重 2.最大化最差用户的权重

In [66]:
m_to_w_score_matrix 

array([[13.65, 14.65, 13.65, ..., 12.65,  7.15, 13.4 ],
       [31.25, 32.9 , 32.25, ..., 28.75, 26.25, 24.75],
       [21.5 , 47.  , 39.75, ..., 28.75, 37.5 , 45.  ],
       ...,
       [22.5 , 37.5 , 52.25, ..., 28.75, 33.75, 32.25],
       [37.5 , 47.5 , 63.75, ..., 46.25, 46.25, 22.25],
       [23.65, 14.65, 13.65, ..., 22.65,  7.15, 10.9 ]])

In [67]:
w_to_m_score_matrix

array([[30.25, 36.25, 24.  , ..., 37.5 , 35.  , 22.75],
       [26.25, 32.9 , 45.4 , ..., 33.75, 43.75, 16.25],
       [25.25, 37.25, 38.5 , ..., 52.25, 45.  , 14.  ],
       ...,
       [ 2.65,  4.65,  7.15, ...,  7.15,  7.15,  2.65],
       [16.25, 28.75, 33.75, ..., 33.75, 33.75, 26.25],
       [20.  , 22.25, 21.25, ..., 19.75, 27.25, 25.  ]])

In [68]:
from mip import *

# 模型一： 最大化匹配总权重
def OneSideMatching1(score_matrix, X, sex_side = 'male'): 
    # X: 每名用户(默认女性)可被推荐的最大次数
    # score_matrix： 用户的分数矩阵，为男性推荐则input为 m_to_w_score_matrix，为女性推荐则input为 w_to_m_score_matrix
    # sex_side： 为哪种性别的用户推荐， male为男性 female为女性
    
    OSM1 = Model(sense=MAXIMIZE, solver_name= CPLEX) # 生成模型 调用Gurobi求解器
    
    # 创建男女全集合，根据数据不同需要变动
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))
    
    # 添加二元变量

    x = [[OSM1.add_var(var_type=BINARY) for j in J] for i in I] # Decision Variable
    
    # 设定目标方程
    
    OSM1.objective = xsum(score_matrix[i][j]*x[i][j] for i in I for j in W_dict[i])
    
    # 根据不同性别添加约束
    if sex_side == 'male':
        for i in I:
            OSM1 += xsum(x[i][j] for j in W_dict[i]) <= X 

        for j in J:
            OSM1 += xsum(x[i][j] for i in M_dict[j]) <= 1 
            
    if sex_side == 'female':
        for i in I:
            OSM1 += xsum(x[i][j] for j in W_dict[i]) <= 1 

        for j in J:
            OSM1 += xsum(x[i][j] for i in M_dict[j]) <= X

    # 计算运行时间， 求解        
    start = time.perf_counter()
    status = OSM1.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start
    
    # 根据不同性别进行解的收集
    solution = []
    
    if sex_side == 'male':
        for i in I:
            for j in J:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    
    if sex_side == 'female':
        for j in J:
            for i in I:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    
    return solution, OSM1.objective_value, runningtime # 匹配结果，模型最优值，运行时间

In [69]:
sol = OneSideMatching1(w_to_m_score_matrix, 4, sex_side = 'female') 
sol[0]

An error occurred while loading the CBC library:	 cannot load library '/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib': dlopen(/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib, 0x0002): tried: '/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (no such file), '/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')).  Additionally, ctypes.util.find_library() did not manage to locate a library called '/Users/zhuanren/anaconda3/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib'



NameError: name 'cbclib' is not defined

In [70]:
from mip import *

# 最大化最小权重
def OneSideMatching2(score_matrix, X, sex_side = 'male'):
    # X: 每名用户(默认女性)可被推荐的最大次数
    # score_matrix： 用户的分数矩阵，为男性推荐则input为 m_to_w_score_matrix，为女性推荐则input为 w_to_m_score_matrix
    # sex_side： 为哪种性别的用户推荐， male为男性 female为女性
    
    
    OSM2 = Model(sense=MAXIMIZE, solver_name=GRB) # 生成模型 调用Gurobi求解器
    
    # 创建男女全集合，根据数据不同需要变动         
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))

    # 添加决策变量
    x = [[OSM2.add_var(var_type=BINARY) for j in J] for i in I]
    z = OSM2.add_var()

    # 设定目标方程
    OSM2.objective = z

    # 根据不同性别添加约束
    if sex_side == 'male':
        for i in I:
            OSM2 += xsum(x[i][j] for j in W_dict[i]) <= X 

        for j in J:
            OSM2 += xsum(x[i][j] for i in M_dict[j]) <= 1 
            OSM2 += xsum(score_matrix[i][j]*x[i][j] for i in M_dict[j]) >= z 
            
    if sex_side == 'female':
        for i in I:
            OSM2 += xsum(x[i][j] for j in W_dict[i]) <= 1 
            OSM2 += xsum(score_matrix[i][j]*x[i][j] for j in W_dict[i]) >= z 
            
        for j in J:
            OSM2 += xsum(x[i][j] for i in M_dict[j]) <= X

    # 计算运行时间， 求解       
    start = time.perf_counter()
    status = OSM2.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start
    
    # 根据不同性别进行解的收集
    solution = []
    
    if sex_side == 'male':
        for i in I:
            for j in J:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    
    if sex_side == 'female':
        for j in J:
            for i in I:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    return solution, OSM2.objective_value, runningtime # 匹配结果，模型最优值，运行时间

In [71]:
sol2 = OneSideMatching2(m_to_w_score_matrix, 2, sex_side = 'male')
sol2[0]

FileNotFoundError: Gurobi not found. Plase check if the
            Gurobi dynamic loadable library is reachable or define
            the environment variable GUROBI_HOME indicating the gurobi
            installation path.
            

array([[13.65, 14.65, 13.65, ..., 12.65,  7.15, 13.4 ],
       [31.25, 32.9 , 32.25, ..., 28.75, 26.25, 24.75],
       [21.5 , 47.  , 39.75, ..., 28.75, 37.5 , 45.  ],
       ...,
       [22.5 , 37.5 , 52.25, ..., 28.75, 33.75, 32.25],
       [37.5 , 47.5 , 63.75, ..., 46.25, 46.25, 22.25],
       [23.65, 14.65, 13.65, ..., 22.65,  7.15, 10.9 ]])

In [80]:
def gale_shapley_modified_v2(m_to_w_score_matrix, w_to_m_score_matrix, M_dict, W_dict):
    num_m, num_w = len(m_to_w_score_matrix), len(w_to_m_score_matrix)
    
    # 初始化每个人的状态为未匹配
    matched_men = [-1] * num_m
    matched_women = [-1] * num_w

    # 男生向女生匹配的顺序
    proposals_order = []
    for m in range(num_m):
        preferences = sorted(W_dict[m], key=lambda w: -m_to_w_score_matrix[m][w])
        new_order = []
        while preferences:
            new_order.append(preferences.pop(0))  # his favorite
            if preferences:
                new_order.append(preferences.pop(len(preferences) // 2))  # middle one
        proposals_order.append(new_order)

    free_men = list(range(num_m))
    while free_men:
        m = free_men.pop(0)
        
        while proposals_order[m]:
            w = proposals_order[m].pop(0)

            # 如果女生w还未匹配或者她更喜欢新的匹配者，并且她也接受这个男生
            if (matched_women[w] == -1 or w_to_m_score_matrix[w][m] > w_to_m_score_matrix[w][matched_women[w]]) and m in W_dict[w]:
                if matched_women[w] != -1:
                    free_men.append(matched_women[w])

                # 更新匹配
                matched_women[w] = m
                matched_men[m] = w
                break

            # 否则男生被拒绝，继续下一个选择
            else:
                continue

    return matched_men

# 输出匹配结果
print(gale_shapley_modified_v2(m_to_w_score_matrix, w_to_m_score_matrix, M_dict, W_dict))

[51, 5, 43, 24, 60, 65, 4, 38, 48, 28, 27, 44, 37, 32, 22, 9, 40, 6, 47, 11, 13, 0, 53, 45, 56, 55, 31, 57, 33, 1, 67, 21, 20, 49, 36, 7, 12, 64, 2, 63, 26, 34, 61, 41, 23, 8, 46, 62, 15, 54, 52, 30, 25, 14, 29, 50, 35, 16, 58, 59, 39, 17, 66, 18, 19, 3, 10, 42]
